In [188]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import requests


# Load Realtime API Stock Data

In [75]:
def get_stock_data(ticker, start_date, end_date):
    api_key = '50dbca4a8576228510cb129fe07db8e9a178bb9b'
    base_url = (f'https://api.tiingo.com/tiingo/daily/{ticker}/prices?')
    payload = {
        'token': api_key, 
        'startDate': start_date,
        'endDate': end_date
    }
#     data_response = requests.get(base_url, params=payload)
#     json_data = data_response.json()
#     df = json_normalize(response)
#     elevations = response.read()
#     data = json.loads(str(response))
#     df = pd.json_normalize(data)
#     df = pd.json_normalize(json_data)
#     df = df[['date', 'close']]
#     df.reset_index(level=0, inplace=True)

    response = requests.get(base_url, params=payload)
    
    
  #  df = pd.DataFrame.from_dict(response, orient="index")
    
    
#     dates = []
#     closes = []
    
#     for json_object in response:
#         date = json_object['date']
#         close = json_object['close']
        
#         dates.append(date)
#         closes.append(close)

#     df = json_normalize(dates)
#     df = pd.DataFrame(closes).transpose()
    return response
    
# panel_data = data.DataReader('INPX', 'google', start_date, end_date)

In [147]:
data_response = get_stock_data('LMND', '2020-06-01', '2020-11-03')
# print(data_response.url)
print(data_response.json())

[{'date': '2020-07-02T00:00:00.000Z', 'close': 69.41, 'high': 70.8, 'low': 49.02, 'open': 50.06, 'volume': 16697700, 'adjClose': 69.41, 'adjHigh': 70.8, 'adjLow': 49.02, 'adjOpen': 50.06, 'adjVolume': 16697700, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-07-06T00:00:00.000Z', 'close': 81.19, 'high': 96.51, 'low': 73.11, 'open': 73.39, 'volume': 13445700, 'adjClose': 81.19, 'adjHigh': 96.51, 'adjLow': 73.11, 'adjOpen': 73.394, 'adjVolume': 13445700, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-07-07T00:00:00.000Z', 'close': 78.79, 'high': 89.38, 'low': 75.0, 'open': 83.8, 'volume': 4599600, 'adjClose': 78.79, 'adjHigh': 89.38, 'adjLow': 75.0, 'adjOpen': 83.8, 'adjVolume': 4599600, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-07-08T00:00:00.000Z', 'close': 68.51, 'high': 79.39, 'low': 68.06, 'open': 79.0, 'volume': 3489400, 'adjClose': 68.51, 'adjHigh': 79.39, 'adjLow': 68.06, 'adjOpen': 79.0, 'adjVolume': 3489400, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020

In [148]:
def create_datasets(ticker, start_date, end_date):
    json_data = data_response.json()
    
    dates = []
    highs = []
    lows = []
    opens = []
    closes = []
    vols = []
    
    for json_object in json_data:
        date = json_object['date']
        high = json_object['high']
        low = json_object['low']
        opened = json_object['open']
        close = json_object['adjClose']
        vol = json_object['volume']
        
        dates.append(date)
        highs.append(high)
        lows.append(low)
        opens.append(opened)
        closes.append(close)
        vols.append(vol)
    
    
    df = pd.DataFrame(list(zip(dates, closes)), columns=['Dates', 'adjClose']) 
    
    return df

In [149]:
# df = pd.read_csv('Data/LMND.csv')
# df.tail()
# # Get the Adjusted Close Price 
# df = df[['Adj Close']] 
# # Take a look at the new data 
# print(df.head())

## Create DataFrame

In [154]:
df = create_datasets('LMND', '2020-06-01', '2020-11-03')
#x_test, y_test = create_datasets('LMND', '2020-06-01', '2020-10-31')

df.tail()

,Dates,adjClose
82,2020-10-28T00:00:00.000Z,49.13
83,2020-10-29T00:00:00.000Z,46.90
84,2020-10-30T00:00:00.000Z,50.29
85,2020-11-02T00:00:00.000Z,52.96
86,2020-11-03T00:00:00.000Z,58.40


In [155]:
# df.tail()
# df['Dates'] = df['Dates'][0][:-14]
# api_df[:-5]
df['Ndates'] = df.Dates.str[:-14]
# df['NDates'] = df.Dates.apply(lambda x: x[:-14])
df.drop('Dates', inplace=True, axis=1)
df['Ndates']

0     2020-07-02
1     2020-07-06
2     2020-07-07
3     2020-07-08
4     2020-07-09
         ...    
82    2020-10-28
83    2020-10-29
84    2020-10-30
85    2020-11-02
86    2020-11-03
Name: Ndates, Length: 87, dtype: object

In [109]:
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        df[label] = content.astype("category").cat.as_ordered()

In [158]:
df['Ndates']

0     2020-07-02
1     2020-07-06
2     2020-07-07
3     2020-07-08
4     2020-07-09
         ...    
82    2020-10-28
83    2020-10-29
84    2020-10-30
85    2020-11-02
86    2020-11-03
Name: Ndates, Length: 87, dtype: object

In [159]:
df['Ndates'] = pd.to_datetime(df['Ndates'])

df['Ndates']

0    2020-07-02
1    2020-07-06
2    2020-07-07
3    2020-07-08
4    2020-07-09
        ...    
82   2020-10-28
83   2020-10-29
84   2020-10-30
85   2020-11-02
86   2020-11-03
Name: Ndates, Length: 87, dtype: datetime64[ns]

In [160]:
# A variable for predicting 'n' days out into the future
forecast_out = 3 #'n=30' days
#Create another column (the target ) shifted 'n' units up
df['Prediction'] = df[['adjClose']].shift(-forecast_out)
#print the new data set
df.tail()

,adjClose,Ndates,Prediction
82,49.13,2020-10-28,52.96
83,46.90,2020-10-29,58.40
84,50.29,2020-10-30,NaN
85,52.96,2020-11-02,NaN
86,58.40,2020-11-03,NaN


In [172]:
df['Ndates'] = pd.to_numeric(df.Ndates, downcast='float')

In [173]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[6.94100000e+01 1.59364796e+18]
 [8.11900000e+01 1.59399362e+18]
 [7.87900000e+01 1.59408007e+18]
 [6.85100000e+01 1.59416638e+18]
 [7.70100000e+01 1.59425283e+18]
 [8.46600000e+01 1.59433914e+18]
 [7.85000000e+01 1.59459835e+18]
 [7.61400000e+01 1.59468480e+18]
 [7.41700000e+01 1.59477125e+18]
 [7.92900000e+01 1.59485756e+18]
 [8.58900000e+01 1.59494401e+18]
 [8.42400000e+01 1.59520322e+18]
 [8.08500000e+01 1.59528967e+18]
 [7.85000000e+01 1.59537598e+18]
 [8.21600000e+01 1.59546243e+18]
 [7.79700000e+01 1.59554874e+18]
 [7.30000000e+01 1.59580795e+18]
 [6.64300000e+01 1.59589440e+18]
 [6.90500000e+01 1.59598085e+18]
 [6.51000000e+01 1.59606716e+18]
 [5.81800000e+01 1.59615361e+18]
 [5.68600000e+01 1.59641282e+18]
 [6.11400000e+01 1.59649927e+18]
 [6.44800000e+01 1.59658558e+18]
 [6.87000000e+01 1.59667203e+18]
 [6.68400000e+01 1.59675834e+18]
 [6.74100000e+01 1.59701755e+18]
 [6.27500000e+01 1.59710400e+18]
 [6.06700000e+01 1.59719045e+18]
 [5.92600000e+01 1.59727676e+18]
 [6.322000

In [174]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(df['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[68.51 77.01 84.66 78.5  76.14 74.17 79.29 85.89 84.24 80.85 78.5  82.16
 77.97 73.   66.43 69.05 65.1  58.18 56.86 61.14 64.48 68.7  66.84 67.41
 62.75 60.67 59.26 63.22 61.48 61.79 61.7  59.86 60.2  66.   61.01 61.13
 58.39 58.18 58.7  56.19 54.7  50.83 47.27 47.18 50.18 52.4  50.12 48.71
 47.15 48.65 51.88 55.24 52.93 54.07 49.87 47.98 50.93 51.25 49.61 49.72
 51.46 51.58 54.   52.44 53.6  60.1  67.81 66.17 66.73 68.   62.41 58.99
 58.95 58.58 57.98 52.53 51.09 51.19 50.73 49.13 46.9  50.29 52.96 58.4 ]


# Split Data into Train & Test

In [175]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Create Regression Models

In [176]:
# random forest regressor 
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_depth=3,random_state=0)
rfr.fit(x_train, y_train)

RandomForestRegressor(max_depth=3, random_state=0)

In [177]:
rfr_confidence = rfr.score(x_test, y_test)
print("rfr confidence", rfr_confidence)

rfr confidence 0.8109306473636954


In [191]:
# Create and train the Support Vector Machine (Regressor)
np.random.seed(42)
svr_rbf = SVR(kernel='poly')
svr_rbf.fit(x_train, y_train)

SVR(kernel='poly')

In [192]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  -0.05941086486178171


In [180]:
# Create and train the Linear Regression  Model
np.random.seed(42)
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression()

In [181]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.6655506956724104


In [182]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[5.02900000e+01 1.60401594e+18]
 [5.29600000e+01 1.60427515e+18]
 [5.84000000e+01 1.60436160e+18]]


In [186]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction[-1])
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction[-1])

rfr_prediction = rfr.predict(x_forecast)
print(rfr_prediction[-1])

54.56202366368393
61.44666666666667
53.37350523335596


In [187]:
lr_prediction

array([49.10654413, 50.7923405 , 54.56202366])